# API Mercado libre (ML_api)

El objetivo es poder armar un dataset con registros de inmuebles publicados en la página de mercado libre.
Los datos de clasidicados son públicos, detallan precios, condiciones de venta/alquiler, características y zonificación del inmueble.
Estos datos publicados por Mercado Libre (ML) en su página http://www.mercadolibre.com.ar son registrados por propietarios o agentes inmobiliarios para su libre consulta.


Para mayor detalle inspeccionar la pág oficial de ML.

In [6]:
# Imports necesarios
import requests
from bs4 import BeautifulSoup
import re
from lxml import etree
import pandas as pd
import json
from pprint import pprint

### API Mercado libre: _propiedades_.

Vamos a hacer peticiones (GET) a la api pública de mercado libre.

La búsqueda puede ser performada por query (consulta global) o category (dirigida a una categoría):

Performs a search by query

    GET
        /sites/MLA/search?q=

Performs a search by category

    GET
        /sites/MLA/search?category=

Performs a search by seller_id

    GET
        /sites/MLA/search?seller_id=

Performs a search by nickname

    GET
        /sites/MLA/search?nickname=

Performs a search by special filters

    GET
        /sites/MLA/search?special_filter=

Performs a search with a specific sorting method

    GET
        /sites/MLA/search?q=ipod&sort=sortId=

Performs a search applying filters

    GET
        /sites/MLA/search?q=ipod&FilterID=FilterValue

En el header tiene los siguientes parámetros {code} que podemos modificar

Utilizaremos el siguiente métos pero inspeccionaremos el funcionamiento de otras búsquedas.

url:

    https://api.mercadolibre.com/sites/MLA/search?category={code}&_PublishedToday_{code}_&limit={code}&offset={code}

Parámetros por categoría:

    **search?category**
    Tipo de propiedades (in la terminación luego de _ aplica a toda operación 'venta', 'alquiler', etc.)
    casas = 'MLA1466' #_242075_242060'
    campos = 'MLA1496_242059'
    cocheras = 'MLA50541_267198'
    departamentos = 'MLA1472_242062'
    depositos_galpones = 'MLA1475_245003'
    locales = 'MLA79242_242065'
    oficinas = 'MLA50538_242067'
    phs = 'MLA105179_242069'
    quintas = 'MLA50547_242070'
    terrenos_lotes = 'MLA1493_242071'

Tiempo de publicación:

    **_PublishedToday_** 
    ('YES', 'NO')
    
paginación

    **limit**
    máx = 50

    **offset**
    número de paginación
    0, 1 , ..., n

códigos de operación

    venta = 242075, alquiler = 242073, alquiler_temporal = 242074

# Propiedades de interes.

In [13]:
# Hay más, estas son las de interes por category id
property_type = {
    'departamentos': 'MLA1472',
    'casas': 'MLA1466',
    'terrenos_lotes': 'MLA1493',
    'phs': 'MLA105179',
    'locales': 'MLA79242',
    'oficinas': 'MLA50538',
    'depositos_galpones': 'MLA1475',
    'cocheras': 'MLA50541',
    'campos': 'MLA1496',
    'quintas': 'MLA50547',
}

# Estuctura de la respuesta a la API

### Data response:

    {
      "site_id": "MLA",
      "country_default_time_zone": "GMT-03:00",
      "paging": {},
      "results": [],
      "sort": {},
      "available_sorts": [],
      "filters": [],
      "available_filters": []
    }

Referencia al citio y la zona.
    
    "site_id": "MLA"
    "country_default_time_zone": "GMT-03:00"

Información de la búsqueda y paginación de la request.
    
    "paging": {
        "total": 219007,            # Total de registros para búsqueda.
        "primary_results": 1000,    # Registros relevantes?
        "offset": 0,                # Inico de búsqueda del [0 ; total/limit) devuelve "_limit_" registros
        "limit": 50,                # Registros devueltos (máx 50) por petición

Informacion requerida.

    "results": []                   # len("results") = limit, siempre que no sea la última página.

# Inspección de prueba

hacemos una petición de un solo registro para conocer la estructura de la información.

In [4]:
# Prueba
#: params
today = 'YES'
# Parámetros mínimos para inspección de estructura.
limit = 1
offs = 0

In [7]:
#'https://api.mercadolibre.com/sites/MLA/search?category=¿?&_PublishedToday_¿?limit=¿?&offset=¿?'
api = f'https://api.mercadolibre.com/sites/MLA/'\
        f'search?category={property_type["departamentos"]}'\
        f'&_PublishedToday_{today}'\
        f'&limit={limit}'\
        f'&offset={offs}'

r = requests.get(api)
r.status_code

200

## Observando la respuesta

Buscamos ver como se estructuran los datos para resgistrar solo aquellos que puedan ser de interes.

In [8]:
data = json.loads(r.content)

type(data), len(data), data.keys()

(dict,
 8,
 dict_keys(['site_id', 'country_default_time_zone', 'paging', 'results', 'sort', 'available_sorts', 'filters', 'available_filters']))

Vemos que el json de respuesta contiene 8 entradas.
Veamos que información trae cada una de ellas.

In [12]:
for key in data:
    # result and avilable_filters: inspección por separado.
    if key != 'results' and key != 'available_filters':
        print(40*'-')
        print('Llave: ',key)
        pprint(data[key])


----------------------------------------
Llave:  site_id
'MLA'
----------------------------------------
Llave:  country_default_time_zone
'GMT-03:00'
----------------------------------------
Llave:  paging
{'limit': 1, 'offset': 0, 'primary_results': 1000, 'total': 301069}
----------------------------------------
Llave:  sort
{'id': 'relevance', 'name': 'Más relevantes'}
----------------------------------------
Llave:  available_sorts
[{'id': 'price_asc', 'name': 'Menor precio'},
 {'id': 'price_desc', 'name': 'Mayor precio'}]
----------------------------------------
Llave:  filters
[{'id': 'category',
  'name': 'Categorías',
  'type': 'text',
  'values': [{'id': 'MLA1472',
              'name': 'Departamentos',
              'path_from_root': [{'id': 'MLA1459', 'name': 'Inmuebles'},
                                 {'id': 'MLA1472', 'name': 'Departamentos'}]}]}]


### 'available_filters'

In [27]:
type(data['available_filters']), len(data['available_filters'])

(list, 32)

aviable_filter contiene 32 variables.

Diccionarios con 4 llaves. Contiene los filtros hablitados en la variable filter.

    ['id', 'name', 'type', 'values']


In [28]:
# Inspección de 'id', 'name', 'type', 'values'.
for i, ty in enumerate (data['available_filters']):
    print(f'{i} -> {ty["id"]}: {ty["name"]}[{ty["type"]}]: {ty["values"]}', end="\n")
    print(50*'-')

0 -> official_store: Tiendas oficiales[text]: [{'id': 'all', 'name': 'Todas las tiendas oficiales', 'results': 34139}, {'id': '2973', 'name': 'Tormes Propiedades', 'results': 89}, {'id': '2310', 'name': 'Salaya Romera', 'results': 1466}, {'id': '2684', 'name': 'Korn Propiedades', 'results': 551}, {'id': '2622', 'name': 'Estudio Yacoub', 'results': 2136}, {'id': '2743', 'name': 'Sistema Coldwell Banker', 'results': 1770}, {'id': '2752', 'name': 'Okeefe', 'results': 550}, {'id': '2938', 'name': 'Girard Propiedades', 'results': 428}, {'id': '2015', 'name': 'Grupo VACCARO', 'results': 62}]
--------------------------------------------------
1 -> state: Ubicación[text]: [{'id': 'TUxBUENBUGw3M2E1', 'name': 'Capital Federal', 'results': 88481}, {'id': 'TUxBUEdSQWU4ZDkz', 'name': 'Bs.As. G.B.A. Norte', 'results': 34406}, {'id': 'TUxBUFNBTmU5Nzk2', 'name': 'Santa Fe', 'results': 32940}, {'id': 'TUxBUENPU2ExMmFkMw', 'name': 'Bs.As. Costa Atlántica', 'results': 32391}, {'id': 'TUxBUEdSQWVmNTVm', '

# Resumen _'_available_filter_'_

Para atomizar el pedido a la API de ML se puede aplicar diferentes filtros concatenados utlizando
* Grupo de la búsqueda:
    /search?q=id o /search?category=id 
* Filtros concatendados:
    &FilterID=FilterValue&FilterID=FilterValue  

ejemplo:
URI:
'https://api.mercadolibre.com/sites/MLA/'

| Búsqueda |  q_id_filter| category_id_filter|
| --- | --- | --- |
| propiedades = Departamento | /search?q=Departamentos | /search?category=MLA1472|

URI:
'https://api.mercadolibre.com/sites/MLA/search?category=MLA1472'

| Filtro | filter key | value | sintax |
| --- | --- | --- | --- |
| Antigüedad = 6 a 25 años | PROPERTY_AGE | [6años-25años] | &PROPERTY_AGE=[6años-25años] |
| Ambientes = 3 y 4 | ROOMS | [3-4] | &ROOMS=[3-4] |
| Ubicación = Capital Federal | state | TUxBUENBUGw3M2E1 | &state=TUxBUENBUGw3M2E1 |


URI:
'https://api.mercadolibre.com/sites/MLA/search?category=MLA1472&PROPERTY_AGE=[6años-25años]&ROOMS=[3-4]&state=TUxBUENBUGw3M2E1'


In [29]:
# # results
len(data['results']), type(data['results'])

(1, list)

In [31]:
# # results
len(data['results'][0]), type(data['results'][0]), data['results'][0].keys()

(41,
 dict,
 dict_keys(['id', 'site_id', 'title', 'seller', 'price', 'prices', 'sale_price', 'currency_id', 'available_quantity', 'sold_quantity', 'buying_mode', 'listing_type_id', 'stop_time', 'condition', 'permalink', 'thumbnail', 'thumbnail_id', 'accepts_mercadopago', 'installments', 'address', 'promotions', 'shipping', 'seller_address', 'seller_contact', 'location', 'attributes', 'original_price', 'category_id', 'official_store_id', 'domain_id', 'catalog_product_id', 'tags', 'order_backend', 'use_thumbnail_id', 'offer_score', 'offer_share', 'match_score', 'winner_item_id', 'melicoin', 'discounts', 'inventory_id']))

In [36]:
# results
for i, k in enumerate (data['results'][0]):
    print(f'{i} -> {k}: {data["results"][0][k]}')#: {ty["title"]}', end="\n")
    print(50*'-')

0 -> id: MLA1211661414
--------------------------------------------------
1 -> site_id: MLA
--------------------------------------------------
2 -> title: Almagro Alquiler 2 Ambientes Espectaculares En Piso Alto !!!!
--------------------------------------------------
3 -> seller: {'id': 156220336, 'permalink': 'http://perfil.mercadolibre.com.ar/ZUKER+INMOBILIARIA', 'registration_date': '2016-02-06T01:38:15.000-04:00', 'car_dealer': False, 'real_estate_agency': True, 'tags': ['real_estate_agency', 'nsm_high', 'from_facebook', 'messages_as_seller'], 'seller_reputation': {'power_seller_status': None, 'level_id': None, 'metrics': {'cancellations': {'period': '365 days', 'rate': 0, 'value': 0}, 'claims': {'period': '365 days', 'rate': 0, 'value': 0}, 'delayed_handling_time': {'period': '365 days', 'rate': 0, 'value': 0}, 'sales': {'period': '365 days', 'completed': 0}}, 'transactions': {'canceled': 0, 'period': 'historic', 'total': 0, 'ratings': {'negative': 0, 'neutral': 0, 'positive': 0},

In [17]:
typer = []
for ty in data['available_filters']:
    typer.append((ty['name'], type(ty), len(ty), list(ty.keys())))

typer

------------------------------
available_filters
[{'id': 'official_store',
  'name': 'Tiendas oficiales',
  'type': 'text',
  'values': [{'id': 'all',
              'name': 'Todas las tiendas oficiales',
              'results': 35922},
             {'id': '2973', 'name': 'Tormes Propiedades', 'results': 148},
             {'id': '2684', 'name': 'Korn Propiedades', 'results': 680},
             {'id': '2622', 'name': 'Estudio Yacoub', 'results': 2448},
             {'id': '2969',
              'name': 'Turdo Estudio Inmobiliario',
              'results': 188},
             {'id': '3013', 'name': 'Gilges Inmobiliaria', 'results': 125},
             {'id': '2695', 'name': 'Goldstein Propiedades', 'results': 611},
             {'id': '2015', 'name': 'Grupo VACCARO', 'results': 62}]},
 {'id': 'state',
  'name': 'Ubicación',
  'type': 'text',
  'values': [{'id': 'TUxBUENBUGw3M2E1',
              'name': 'Capital Federal',
              'results': 86899},
             {'id': 'TUxBUENPU2ExMm

In [13]:
pd.json_normalize(data, record_path = ['results'])

,id,site_id,title,price,sale_price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,...,location.neighborhood.id,location.neighborhood.name,location.city.id,location.city.name,location.state.id,location.state.name,location.country.id,location.country.name,location.latitude,location.longitude
0,MLA1203712001,MLA,Depto 2 Amb Centrico!!- Ramos Mejia,38000,None,ARS,1,0,classified,gold_premium,...,TUxBQlJBTTE3Mzla,Ramos Mejía,TUxBQ0xBTWF0YW56,La Matanza,TUxBUEdSQWVmNTVm,Bs.As. G.B.A. Oeste,AR,Argentina,-34.6455,-58.57164


In [38]:
len(data['results'])

2

In [40]:
data = requests.get(api).text

In [41]:
json.loads(data)

{'site_id': 'MLA',
 'country_default_time_zone': 'GMT-03:00',
 'paging': {'total': 299418, 'primary_results': 1000, 'offset': 0, 'limit': 2},
 'results': [{'id': 'MLA1203712001',
   'site_id': 'MLA',
   'title': 'Depto 2 Amb Centrico!!- Ramos Mejia',
   'seller': {'id': 167144578,
    'permalink': 'http://perfil.mercadolibre.com.ar/INMOBILIARIA+GILGES',
    'registration_date': '2014-09-20T08:41:24.000-04:00',
    'car_dealer': False,
    'real_estate_agency': True,
    'tags': ['real_estate_agency',
     'brand',
     'nsm_high',
     'credits_profile',
     'credits_priority_3',
     'messages_as_seller'],
    'seller_reputation': {'power_seller_status': None,
     'level_id': None,
     'metrics': {'cancellations': {'period': '365 days',
       'rate': 0,
       'value': 0},
      'claims': {'period': '365 days', 'rate': 0, 'value': 0},
      'delayed_handling_time': {'period': '365 days', 'rate': 0, 'value': 0},
      'sales': {'period': '365 days', 'completed': 0}},
     'transact

In [42]:
pd.DataFrame.from_dict(json.loads(data)['results'])

,id,site_id,title,seller,price,prices,sale_price,currency_id,available_quantity,sold_quantity,...,tags,order_backend,use_thumbnail_id,offer_score,offer_share,match_score,winner_item_id,melicoin,discounts,inventory_id
0,MLA1203712001,MLA,Depto 2 Amb Centrico!!- Ramos Mejia,"{'id': 167144578, 'permalink': 'http://perfil....",38000,"{'id': 'MLA1203712001', 'prices': [{'id': '1',...",None,ARS,1,0,...,"[good_quality_picture, good_quality_thumbnail]",1,True,None,None,None,None,None,None,None
1,MLA1205638836,MLA,Alquiler Departamento Palermo S. Ortiz Y Cervi...,"{'id': 631374458, 'permalink': 'http://perfil....",75000,"{'id': 'MLA1205638836', 'prices': [{'id': '2',...",None,ARS,1,0,...,"[good_quality_picture, good_quality_thumbnail]",2,True,None,None,None,None,None,None,None


In [84]:
pd.json_normalize(data, record_path = ['results'])

,id,site_id,title,price,sale_price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,...,location.neighborhood.id,location.neighborhood.name,location.city.id,location.city.name,location.state.id,location.state.name,location.country.id,location.country.name,location.latitude,location.longitude
0,MLA1203712001,MLA,Depto 2 Amb Centrico!!- Ramos Mejia,38000,None,ARS,1,0,classified,gold_premium,...,TUxBQlJBTTE3Mzla,Ramos Mejía,TUxBQ0xBTWF0YW56,La Matanza,TUxBUEdSQWVmNTVm,Bs.As. G.B.A. Oeste,AR,Argentina,-34.6455,-58.57164


In [ ]:
for offset in range(0,paginators):
    url = self.ml_url + self.objeto.mercadolibre_id + '&_PublishedToday_YES&limit=50&offset=' + str(offset*50)
    jsdata = self.request_get(url)
    if(jsdata is not None): self.items = self.items + jsdata['results']
self.adapt()